In [19]:
import math
import random
import sys
import operator

import pandas as pd
import numpy as np

In [20]:
L = 5
N = 6
ITERATIONS = 10
MUTATION_CHANCE = 0.1
GENE_MASK = []

In [21]:
def fitness(individ):
    return L

In [22]:
def get_max_num_for_bits(bits):
    max_bits = math.log2(sys.maxsize * 2 + 1)
    return int(sys.maxsize * 2 + 1) >> int(max_bits - bits)

In [23]:
def uniform_individ():
    return random.randint(0, get_max_num_for_bits(L))

def zero_individ():
    return 0


def initial_pool(n, option="uniform"):
    pool = []
    for i in range(0, n):
        if option is "uniform":
            pool.append(random.randint(0, get_max_num_for_bits(L)))
        elif option is "zeroes":
            pool.append(0)
        elif option is "standart":
            pool.append(0)
        else:
            raise SyntaxError("no such option for initial pool: {option}")
    return pool

<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-23-9dd0e74aac9c>:11: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if option is "uniform":
<ipython-input-23-9dd0e74aac9c>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif option is "zeroes":
<ipython-input-23-9dd0e74aac9c>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif option is "standart":


In [24]:
def get_percentile(probs, throw):
    i = 0
    while throw >= probs[i]:
        throw -= probs[i]
        i += 1
    return i

def rws(pool, probs, n):
    new_pool = []
    for i in range(0, n):
        throw = random.random()
        perc = get_percentile(probs, throw)
        new_pool.append(pool[perc])
    return new_pool

In [25]:
def should_mutate() -> bool:
    return random.random() <= MUTATION_CHANCE

def mutate(individual):
    for i in range(0, L):
        if should_mutate():
            individual = individual ^ (1 << i)
    return individual

In [26]:
def is_bit_set(num, i):
    return num & (1 << i) > 0

In [27]:
def get_ith(lst, i):
    return lst[i] if -len(lst) <= i < len(lst) else 0

def count_bits(num, mask=[]):
    count = 0
    for i in range(0, L):
        if get_ith(mask, i) == 0 and is_bit_set(num, i):
            count += 1
    return count

count_bits(5)


2

In [28]:
def hamming_distance(n1, n2) -> int: 
    x = n1 ^ n2  
    return count_bits(x) 

print(hamming_distance(2, 5) == 3)
print(hamming_distance(5, 5) == 0)
print(hamming_distance(0, 31) == 5)

True
True
True


In [29]:
def wild_type(population) -> int:
    wild_type = 0
    for i in range(0, L):
        ones = 0
        for individ in population:
            if is_bit_set(individ, i):
                ones += 1
        mode = round(float(ones) / len(population))
        wild_type |= (mode << i)
    return wild_type

t1 = wild_type([int('11', 2), int('11', 2)]) # 11 -> 3
t2 = wild_type([int('110', 2), int('101', 2), int('000', 2)]) # 100 -> 4

print(t1, bin(t1))
print(t2, bin(t2))



3 0b11
4 0b100


In [30]:
def gen_columns(length):
    columns = ["Кількість особин у популяції"]

    for i in range(0, length+1):
        columns.append("Відстань " + str(i))
    
    columns.extend([
                    "Відстань: математичне сподівання",
                    "Відстань: середнє квадратичне відхилення",
                    "Відстань: мода",
                    "Відсоток поліморфних нейтральних генів у популяції з 1 заміною",
                    "Відсоток поліморфних нейтральних генів у популяції з кількома замінами",
                    "«Дикий тип»: % поліморфних генів",
                    "«Дикий тип»: кількість поліморфних генів",
                    "Модуль різниці середнього здоров’я в популяції від оптимального",
                    "Модуль різниці найкращого здоров’я в популяції від оптимального"
    ])

    return columns

stat_columns = gen_columns(L)

In [31]:
def get_pair_distandes(population):
    grouped_distances = dict()
    total = 0
    for row in range(0, len(population)):
        for col in range(0, len(population)):
            if row == col:
                break
            distance = hamming_distance(population[row], population[col])
            grouped_distances[distance] = grouped_distances.get(distance, 0) + 1
            total += 1

    return grouped_distances, total

def get_population_stats(population, fits):
    line = dict()
    line["Кількість особин у популяції"] = [len(population)]
    pairs, total_pairs = get_pair_distandes(population)
    for i in range(0, L+1):
        line[f"Відстань {i}"] = [pairs.get(i, 0) / total_pairs]

    mean = sum([p[0] * (p[1] / float(total_pairs)) for p in pairs.items()])
    line["Відстань: математичне сподівання"] = [mean]

    std = math.sqrt(sum([pow(p[0] - mean, 2) * (p[1] / float(total_pairs)) for p in pairs.items()]))
    line["Відстань: середнє квадратичне відхилення"] = [std]

    mode = max(pairs.items(), key=operator.itemgetter(1))[0]
    line["Відстань: мода"] = [mode]

    all_set_bits = 0
    for locus in range(0, L):
      for ind in population:
        if is_bit_set(ind, locus):
          all_set_bits |= (mode << locus)
          continue

    polymorph_bits = count_bits(all_set_bits, GENE_MASK) / float(L)

    line["Відсоток поліморфних нейтральних генів у популяції з 1 заміною"] = polymorph_bits
    line["Відсоток поліморфних нейтральних генів у популяції з кількома замінами"] = polymorph_bits

    wild = wild_type(population)
    wild_polymorph = count_bits(wild, GENE_MASK)

    line["«Дикий тип»: % поліморфних генів"] = wild_polymorph / float(L)
    line["«Дикий тип»: кількість поліморфних генів"] = wild_polymorph

    module_mean = abs((sum(fits) / len(fits)) - fitness(0))
    module_best = abs(max(fits) - fitness(0))

    line["Модуль різниці середнього здоров’я в популяції від оптимального"] = module_mean
    line["Модуль різниці найкращого здоров’я в популяції від оптимального"] = module_best

    return pd.DataFrame(line, columns=stat_columns)


In [32]:
def format_bin(lst):
    return list(map(lambda x: format(x, f'0{L}b'), lst))

In [33]:
#Refactor to be generic
def simulation():
    pool = initial_pool(N, "zeroes")
    df = pd.DataFrame(columns=stat_columns)

    for i in range(0, ITERATIONS):

        print("Iteration ", i)
        fits = []
        for ind in pool:
            fit = fitness(ind)
            fits.append(fit)
    
        print("Fitnesses: ", fits)
        total_fit = sum(fits)
    
        probs = []
        for fit in fits:
            prob = float(fit) / total_fit
            probs.append(prob)

        stats = get_population_stats(pool, fits)
        df = df.append(stats, ignore_index=True)

        print("Probabilities: ", probs)

        new_pool = rws(pool, probs, N)
        print("Selected pool:  ", format_bin(new_pool))
        for i in range(0, len(new_pool)):
            new_pool[i] = mutate(new_pool[i])
        print("After mutation: ", format_bin(new_pool))

        pool = new_pool
    return df

In [34]:
#Stub
def init_pool(L, growth_option):
  pool = []
  pool.append(0)
  return pool

#Stub
def calc_fitness(pool, selection_option):
  fitness = []
  fitness.append(10)
  return fitness

#Stub
def get_successor_pool_size(growth_option, iteration):
  return 1

def simulate(L, growth_option, Px, selection_option):
  pool = init_pool(L, growth_option)
  for iteration in range(0, ITERATIONS):
    fitness = calc_fitness(pool, selection_option)
#    successor_pool_size = get_successor_pool_size(growth_option, iteration)
#    predecessor_pool = select_predecessor_pool(pool, fitness, selection_option, successor_pool_size)
#    successor_pool = mutate(predecessor_pool, Px)






In [35]:
def play_simulation():
    df = simulation()
    return df


In [36]:
play_simulation()

Iteration  0
Fitnesses:  [5, 5, 5, 5, 5, 5]
Probabilities:  [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
Selected pool:   ['00000', '00000', '00000', '00000', '00000', '00000']
After mutation:  ['00000', '00000', '00000', '00010', '00000', '00000']
Iteration  1
Fitnesses:  [5, 5, 5, 5, 5, 5]
Probabilities:  [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
Selected pool:   ['00000', '00000', '00000', '00000', '00000', '00000']
After mutation:  ['00100', '01010', '00000', '00000', '01000', '00000']
Iteration  2
Fitnesses:  [5, 5, 5, 5, 5, 5]
Probabilities:  [0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
Selected pool:   ['00000', '01000', '00000', '01010', '00000', '00000']
After mutation:  ['00000', '01000', '10000', '00010', '01010', '00110']
Iterati

,Кількість особин у популяції,Відстань 0,Відстань 1,Відстань 2,Відстань 3,Відстань 4,Відстань 5,Відстань: математичне сподівання,Відстань: середнє квадратичне відхилення,Відстань: мода,Відсоток поліморфних нейтральних генів у популяції з 1 заміною,Відсоток поліморфних нейтральних генів у популяції з кількома замінами,«Дикий тип»: % поліморфних генів,«Дикий тип»: кількість поліморфних генів,Модуль різниці середнього здоров’я в популяції від оптимального,Модуль різниці найкращого здоров’я в популяції від оптимального
0,6,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0,0.0,0
1,6,0.666667,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333,0.471405,0,0.0,0.0,0.0,0,0.0,0
2,6,0.200000,0.466667,0.266667,0.066667,0.000000,0.000000,1.200000,0.832666,1,0.6,0.6,0.0,0,0.0,0
3,6,0.000000,0.400000,0.400000,0.200000,0.000000,0.000000,1.800000,0.748331,1,0.8,0.8,0.0,0,0.0,0
4,6,0.200000,0.466667,0.266667,0.066667,0.000000,0.000000,1.200000,0.832666,1,0.6,0.6,0.0,0,0.0,0
5,6,0.133333,0.266667,0.400000,0.066667,0.133333,0.000000,1.800000,1.166190,2,0.8,0.8,0.0,0,0.0,0
6,6,0.066667,0.066667,0.400000,0.333333,0.000000,0.133333,2.533333,1.257864,2,0.8,0.8,0.2,1,0.0,0
7,6,0.066667,0.333333,0.133333,0.200000,0.200000,0.066667,2.333333,1.445299,1,1.0,1.0,0.2,1,0.0,0
8,6,0.000000,0.000000,0.733333,0.000000,0.266667,0.000000,2.533333,0.884433,2,0.8,0.8,0.6,3,0.0,0
9,6,0.000000,0.066667,0.533333,0.200000,0.133333,0.066667,2.600000,1.019804,2,0.8,0.8,0.6,3,0.0,0
